<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

%load_ext autoreload
# "1" means: always reload modules marked with "%aimport"
# "2" means: always reload all modules except those marked with "%aimport"
%autoreload 1

# from pathlib import Path
# import sys
# import os

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook

from postal.expand import expand_address
# from postal.parser import parse_address

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

plt.rc('figure', figsize=(8.0, 6.0))

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
# np.set_printoptions(precision=4, linewidth=100)

In [2]:
filename_inspect = 'data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv'
filename_rest = 'data/restaurants_nyc_dedupe.csv'

df_rest = pd.read_csv(filename_rest)
df_inspect = pd.read_csv(filename_inspect)

In [3]:
df_rest['address'] = df_rest['address'].str.replace('.', '')
df_rest['address'] = df_rest['address'].str.replace(f', NY, ', ', New York, ', case=True, regex=False)

# df_rest['name'] = df_rest['name'].str.title()
# df_rest['address'] = df_rest['address'].str.title()

In [4]:
# drop some records with nulls
df_inspect = df_inspect.dropna(subset=['DBA', 'BUILDING', 'STREET', 'BORO', 'ZIPCODE'], how='any')

# remove duplicated rows to end up with one row per establishment
# (originally, each row is a report, not an establishment)
df_inspect = df_inspect.drop_duplicates(subset=['CAMIS'], keep='first')

df_inspect['STATE'] = 'New York'

df_inspect['BUILDING'] = df_inspect['BUILDING'].astype(str)
df_inspect['ZIPCODE'] = df_inspect['ZIPCODE'].astype(int).astype(str)

df_inspect = df_inspect.rename(columns={'DBA': 'name'})

df_inspect['name'] = df_inspect['name'].str.title()
df_inspect['BORO'] = df_inspect['BORO'].str.title()
df_inspect['STREET'] = df_inspect['STREET'].str.title().str.replace('.', '')

In [5]:
# df_rest['address'].str.split(',', n=1).str[0].str.rsplit(' ').str[-1].value_counts().index.tolist()

to_replace = {
    'St': 'Street',
    'Ave': 'Avenue',
    'Ln': 'Lane',
    'Pl': 'Place',
    'Blvd': 'Boulevard',
    'Ste': 'Suite',
    'Cir': 'Circle',
    'Fl': 'Floor',
    'Plz': 'Plaza',
    'Tpke': 'Turnpike',
    'Rd': 'Road',
    'Frnt': 'Front',
    'Hwy': 'Highway',
    'Sq': 'Square',
    'Expy': 'Expressway',
    'Ctr': 'Center',
    'Ter': 'Terrace',
    'Dr': 'Drive',
    'Bld': 'Building',
}

for k, v in to_replace.items():
    df_rest['address'] = df_rest['address'].str.replace(rf'\b{k}\b', v, case=False, regex=True)
    df_inspect['STREET'] = df_inspect['STREET'].str.replace(rf'\b{k}\b', v, case=False, regex=True)

In [6]:
df_inspect['address'] = df_inspect['BUILDING'].str.cat(
    df_inspect['STREET'], sep=' ').str.cat(
    df_inspect['BORO'], sep=', ').str.cat(
    df_inspect['STATE'], sep=', ').str.cat(
    df_inspect['ZIPCODE'], sep=', ')

df_inspect['address'] = df_inspect['address'].str.replace('.', '')

In [7]:
df_rest.to_csv('data/restaurants_nyc_dedupe_processed.csv')
df_inspect.to_csv('data/DOHMH_New_York_City_Restaurant_Inspection_Results_processed.csv')

In [8]:
print(df_rest.shape)

df_rest.sample(n=5)

(9979, 2)


,address,name
4919,"132 E 56th Street, New York, New York, 10022",Pizza by Certe
7377,"1431 Prospect Avenue, Bronx, New York, 10456",Mr Grill Deli
5520,"3411 Jerome Avenue, Bronx, New York, 10467",Popeye's
9261,"445 Albee Square, Brooklyn, New York, 11201",Eight Turn Crepe (Dekalb Market Hall)
2892,"47 W 55th Street, New York, New York, 10019",Dim Sum Palace


In [9]:
print(df_inspect.shape)

df_inspect.sample(n=5)

(26067, 20)


,CAMIS,name,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,STATE,address
2303,50056325,Gordo'S Cantina,Queens,2411,Queens Plaza N,11101,6463225227,Mexican,02/21/2018,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,22.0,NaN,NaN,02/06/2019,Pre-permit (Operational) / Initial Inspection,New York,"2411 Queens Plaza N, Queens, New York, 11101"
26458,50069021,Way Point,Manhattan,65,Ludlow Street,10002,9144837042,CafÃ©/Coffee/Tea,10/23/2017,Violations were cited in the following area(s).,10I,"Single service item reused, improperly stored,...",Not Critical,13.0,A,10/23/2017,02/06/2019,Pre-permit (Operational) / Re-inspection,New York,"65 Ludlow Street, Manhattan, New York, 10002"
21870,40585479,Casanova Restaurant,Brooklyn,338,Mc Guiness Boulevard,11222,7183890990,Italian,07/18/2017,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,13.0,A,07/18/2017,02/06/2019,Cycle Inspection / Initial Inspection,New York,"338 Mc Guiness Boulevard, Brooklyn, New York, ..."
17784,50055159,Masakari Fusion,Brooklyn,1631,Sheepshead Bay Road,11235,7183323899,Japanese,11/20/2018,Violations were cited in the following area(s).,06F,Wiping cloths soiled or not stored in sanitizi...,Critical,12.0,A,11/20/2018,02/06/2019,Cycle Inspection / Initial Inspection,New York,"1631 Sheepshead Bay Road, Brooklyn, New York, ..."
15499,40401093,French Roast,Manhattan,2340,Broadway,10024,2127991533,French,09/05/2017,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,12.0,A,09/05/2017,02/06/2019,Cycle Inspection / Re-inspection,New York,"2340 Broadway, Manhattan, New York, 10024"


In [10]:
# expand_address('11 W 32nd St Frnt 1 # 6, New York, NY, 10001')

# fuzz.ratio(row_rest['address_expand'][0], '31-91 21st st astoria new york 11106')

In [11]:
df_rest['address_expand'] = df_rest['address'].apply(expand_address)
df_inspect['address_expand'] = df_inspect['address'].apply(expand_address)

In [12]:
# compare to 0th expanded address in df_inspect
options = df_inspect['address_expand'].apply(lambda x: x[0])

df_inspect['fuzzy_matched'] = False
df_rest['fuzzy_matched'] = False
df_rest['matched_address'] = np.nan
df_rest['matched_name'] = np.nan
df_rest['matched_index'] = np.nan

scorer = fuzz.ratio
score_cutoff = 95

for idx_rest, row_rest in tqdm_notebook(df_rest.iterrows(), total=df_rest.shape[0]):
    result = process.extractOne(row_rest['address_expand'][0], options,
                                scorer=scorer,
                                score_cutoff=score_cutoff)
    if result:
        df_rest.loc[idx_rest, 'fuzzy_matched'] = True
        df_rest.loc[idx_rest, 'matched_address'] = df_inspect.loc[result[2], 'address']
        df_rest.loc[idx_rest, 'matched_name'] = df_inspect.loc[result[2], 'name']
        df_rest.loc[idx_rest, 'matched_index'] = result[2]
        # print('=' * 30)
        # print(result)
        
        # remove this entry from consideration
        options = options[~options.index.isin([result[2]])]


In [13]:
df_rest.to_csv('data/restaurants_nyc_dedupe_processed_fuzzymatchTo0.csv')

In [ ]:
df_inspect['fuzzy_matched'] = False
df_rest['fuzzy_matched'] = False
df_rest['matched_address'] = np.nan
df_rest['matched_name'] = np.nan
df_rest['matched_index'] = np.nan

scorer = fuzz.ratio
score_cutoff = 95

for idx_rest, row_rest in tqdm_notebook(df_rest.iterrows(), total=df_rest.shape[0]):
    for idx_insp, row_insp in df_inspect.loc[~df_inspect['fuzzy_matched']].iterrows():
        # compare to all expanded addresses in df_inspect
        result = process.extractOne(row_rest['address_expand'][0], row_insp['address_expand'],
                                    scorer=scorer,
                                    score_cutoff=score_cutoff)
        if result:
            df_inspect.loc[idx_insp, 'fuzzy_matched'] = True
            df_rest.loc[idx_rest, 'fuzzy_matched'] = True
            df_rest.loc[idx_rest, 'matched_address'] = row_insp['address']
            df_rest.loc[idx_rest, 'matched_name'] = row_insp['name']
            df_rest.loc[idx_rest, 'matched_index'] = idx_insp
            # print('=' * 30)
            # print(idx_insp)
            # print(result)
            break

#         # compare to 0th expanded address in df_inspect
#         if scorer(row_rest['address_expand'][0], row_insp['address_expand'][0]) >= score_cutoff:
#             df_inspect.loc[idx_insp, 'fuzzy_matched'] = True
#             df_rest.loc[idx_rest, 'fuzzy_matched'] = True
#             df_rest.loc[idx_rest, 'matched_address'] = row_insp['address']
#             df_rest.loc[idx_rest, 'matched_name'] = row_insp['name']
#             df_rest.loc[idx_rest, 'matched_index'] = idx_insp
#             # print('=' * 30)
#             # print(idx_insp)
#             # print(result)
#             break

In [ ]:
df_rest.to_csv('data/restaurants_nyc_dedupe_processed_fuzzymatchToAll.csv')